# kostatiniyye 학습

## 라이브러리 로드

In [ ]:
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data import DataLoader

# 자체 라이브러리
from utils import model_controler, custom_logger
from models.kostantiniyye import Kostantiniyye

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## 데이터 로드
+ 각각의 촉감에 대해 아래 touch변수를 조절하여 각각 실행할 것 

In [ ]:
touch = 'softness'

In [ ]:
train = torch.load(f'./dataset/{touch}_train.pth')
test = torch.load(f'./dataset/{touch}_test.pth')

train_loader = DataLoader(train, batch_size=128, shuffle=True)
test_loader = DataLoader(test, batch_size=128, shuffle=False)

## 모델 로드

In [ ]:
model = Kostantiniyye(device=device)
model.load_state_dict(torch.load(f'.checkpoints/pretrained_kostantiniyye.pth'), strict = False)

In [ ]:
model_controler.freeze_model(model, exception=
             ['attention', 'normalize']) # 인코더 노멀라이제이션은 다른 촉감에서 가져옴
model.classifier.mobilenet.classifier[3].requires_grad = True
model.classifier.mobilenet.classifier[3].requires_grad 

In [ ]:
model.to(device)

## 하이퍼 파라미터 세팅

In [ ]:
logger = custom_logger.get_logger(f'kostatiniyye_{touch}')

In [ ]:
# 손실 함수 및 최적화 함수 정의
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
num_epochs = 10
test_step = 1

In [ ]:
results = []

## 학습 루프

In [ ]:
logger.info(f'start new run lr:{lr}-------------')
result = model_controler.run_train_test_2_input(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    best_test_loss = 999.,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    test_step=test_step,
    logger = logger,
    device = device,
    path = f'.checkpoints/best_kostantiniyye_{touch}.pth')
results.append(result)

## 결과 저장

In [ ]:
model_controler.save_result(results=result, path = f'kostantiniyye_{touch}.pkl')